In [10]:
# import splunklib as splunk
import splunklib.client as client
import splunklib.results as results
import urllib.parse
from pandas.io.sql import SQLTable

def _execute_insert(self, conn, keys, data_iter):
    data = [dict(zip(keys, row)) for row in data_iter]
    conn.execute(self.table.insert().values(data))

SQLTable._execute_insert = _execute_insert

import pandas as pd
import vertica_python
import sqlalchemy as sa
import pyodbc
import sys
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.types import VARCHAR
from sqlalchemy.types import Integer
from time import sleep
from pandas.io.sql import SQLTable
import psycopg2
import io #
import tableauserverclient as TSC

print("Connect to Splunk...\n")

HOST = "oprdfdshd300.corp.intuit.net"    #connection param
PORT = 8089
USERNAME = "douaknine" # your username
PASSWORD = 'Logorn12' # your password
APP = "ILAggregation"

service = client.connect(
    host=HOST,
    port=PORT,
    username=USERNAME,
    password=PASSWORD,
    app=APP)


print("Running Splunk query...\n")


mysavedsearch = service.saved_searches["data_dashboards"] #My Dashboard
job = mysavedsearch.dispatch()


while True:
    while not job.is_ready():
        pass
    stats = {"isDone": job["isDone"],
             "doneProgress": float(job["doneProgress"])*100,
              "scanCount": int(job["scanCount"]),
              "eventCount": int(job["eventCount"]),
              "resultCount": int(job["resultCount"])}

    status = ("\r%(doneProgress)03.1f%%   %(scanCount)d scanned   "
              "%(eventCount)d matched   %(resultCount)d results") % stats

    sys.stdout.write(status)
    sys.stdout.flush()
    if stats["isDone"] == "1":
        sys.stdout.write("\n Splunk query Done!\n\n")
        break
    sleep(2)

# Create a Dataframe
d = []            
for result in results.ResultsReader(job.results(count = 0)):
    d.append(result)

job.cancel()   
sys.stdout.write('\n')

df = pd.DataFrame(d)  # main df
print("DataFrame Ready...\n")

print("Field astype config\n")

df['TICKET_ID']= df['TICKET_ID'].astype(int)
df['FAILURE_COUNT']= df['FAILURE_COUNT'].astype(int)
df['CREATED_ERROR_CODE']= df['CREATED_ERROR_CODE'].astype(int)
df['TICKET_CREATION_DATE']= pd.to_datetime(df['TICKET_CREATION_DATE'])
df['LAST_MODIFIED_DATE']= pd.to_datetime(df['LAST_MODIFIED_DATE'])
df['TAT_WITH_EXCLUSION_IN_HOURS']= df['TAT_WITH_EXCLUSION_IN_HOURS'].astype(float)
df['TAT_WITHOUT_EXCLUSION_IN_HOURS']= df['TAT_WITHOUT_EXCLUSION_IN_HOURS'].astype(float)
df['EFFORT_IN_HOURS']= df['EFFORT_IN_HOURS'].astype(float)
df['ASSIGNED_TEAM_ID']= df['ASSIGNED_TEAM_ID'].astype(int)

df = df[['Owner', 'TICKET_ID', 'FAILURE_COUNT', 'TICKET_TYPE', 'SCRIPTNAME',
       'INSTITUTION', 'CREATED_ERROR_CODE', 'STATUS', 'TAG_NAME',
       'TICKET_CREATION_DATE', 'LAST_MODIFIED_DATE', 'PRODUCT_NAME',
       'COUNTRY_CODE', 'TAT_WITH_EXCLUSION_IN_HOURS',
       'TAT_WITHOUT_EXCLUSION_IN_HOURS', 'INCIDENT_NUMBER', 'Team',
       'ASSIGNEDTO', 'EFFORT_IN_HOURS', 'SLA_MET', 'TICKET_DESC', 'manager',
       'CHANNEL_TYPE_NAME', 'ERROR_CATEGORY', 'Error_Description'
       , 'req_uri', 'rootCauseName', 'ticketRootCauseId','ASSIGNED_TEAM_ID']]

con1 = vertica_python.connect(host='vertica.data-lake-prd.a.intuit.com', port=5433, dbname='IDEASHARED', user='douaknine', password='Logorn12')
cursor = con1.cursor()

cursor.execute("""truncate table FDP_ANALYTICS_WS.fdp_dashboards_temp_with_SF;""")
copy_str = """COPY FDP_ANALYTICS_WS.fdp_dashboards_temp_with_SF(
Owner,
TICKET_ID,
FAILURE_COUNT,
TICKET_TYPE,
SCRIPTNAME,
INSTITUTION,
CREATED_ERROR_CODE,
STATUS,
TAG_NAME,
TICKET_CREATION_DATE,
LAST_MODIFIED_DATE,
PRODUCT_NAME,
COUNTRY_CODE,
TAT_WITH_EXCLUSION_IN_HOURS,
TAT_WITHOUT_EXCLUSION_IN_HOURS,
INCIDENT_NUMBER,
Team,
ASSIGNEDTO,
EFFORT_IN_HOURS,
SLA_MET,
TICKET_DESC,
manager,
CHANNEL_TYPE_NAME,
ERROR_CATEGORY,
Error_Description,
req_uri,
rootCauseName,
ticketRootCauseId,
ASSIGNED_TEAM_ID)FROM STDIN DELIMITER ','"""

stream = io.StringIO()
df.to_csv(stream, sep=",",index=False, header=False)

print("Stream init...\n")
stream.seek(0) 


with con1.cursor() as cursor:
    cursor.copy(copy_str,stream.getvalue())
con1.commit()

print("fdp_dashboards_temp_with_SF ...ok\n")

cursor.execute("""select count(case when a.TICKET_ID is null then 1 else null end) new_record,
        count(case when b.TICKET_ID is null then 1 else null end) archived_record,
        count(case when a.TICKET_ID is not null and b.TICKET_ID is not null then 1 else null end) overlap,
        count(*) total_records
from FDP_ANALYTICS_WS.fdp_dashboards_with_SF a full outer join
        FDP_ANALYTICS_WS.fdp_dashboards_temp_with_SF b on a.TICKET_ID = b.TICKET_ID;""")


q1row = cursor.fetchall()
print('new_record - archived_record - overlap - total_records')
print(q1row)


print('\n')

cursor.execute("""drop table FDP_ANALYTICS_WS.fdp_dashboards_backup_with_SF;""")
#Drop if right OK

cursor.execute("""select *
into FDP_ANALYTICS_WS.fdp_dashboards_backup_with_SF
from FDP_ANALYTICS_WS.fdp_dashboards_with_SF a
;""")
cursor.execute("""delete from FDP_ANALYTICS_WS.fdp_dashboards_with_SF where TICKET_ID in 
(
select distinct a.TICKET_ID
from FDP_ANALYTICS_WS.fdp_dashboards_with_SF a full outer join
        FDP_ANALYTICS_WS.fdp_dashboards_temp_with_SF b on a.TICKET_ID = b.TICKET_ID
where a.TICKET_ID is not null and b.TICKET_ID is not null
);""")
cursor.execute("""insert into FDP_ANALYTICS_WS.fdp_dashboards_with_SF
select *
from FDP_ANALYTICS_WS.fdp_dashboards_temp_with_SF
;""")
con1.commit()

print("fdp_dashboards_backup_with_SF ...ok\n")
print("fdp_dashboards_with_SF ...ok\n")


print("DONE :)\n")

print('Refresh Tableau Server...\n')


tableau_auth = TSC.TableauAuth('douaknine', 'Logorn12', site_id = 'FDS')
server = TSC.Server('https://tableau.a.intuit.com',use_server_version = True)
server.auth.sign_in(tableau_auth)
print('connection made\n')
print(server.version)
server.workbooks.refresh(workbook_id='ac076c97-73bb-4c80-95b3-924494e12c44')
server.workbooks.refresh(workbook_id='37aff87d-4dca-4f23-bba5-975f69daefe4')
server.workbooks.refresh(workbook_id='bf33b515-30c6-46df-9019-6940e47b81f8')
server.workbooks.refresh(workbook_id='02451361-b8ac-4542-a3c0-dca6e054e202')
print('refresh complete\n')

server.auth.sign_out()
print('connection closed\n')



print("DONE :)")



Connect to Splunk...

Running Splunk query...

67.7%   2545 scanned   2545 matched   4139 results
 Splunk query Done!


DataFrame Ready...

Field astype config

Stream init...

fdp_dashboards_temp_with_SF ...ok

new_record - archived_record - overlap - total_records
[[6, 64223, 4133, 68362]]


fdp_dashboards_backup_with_SF ...ok

fdp_dashboards_with_SF ...ok

DONE :)

Refresh Tableau Server...

connection made

2.8
refresh complete

connection closed

DONE :)
